# Load project YAML from - Git, Zip, Tar source

After you create your project YAML you can simply load that project and run, build and deploy your functions and workflows.
Please run the Create CI/CD Automation Using - Git source example notebook before you are running this one

In this session you would learn how to:
* Load or Create a new project for remote URL
* Get a function object 
* Run a function
* Build a function
* Deploy a function
* Run a workflow

Install mlrun - if does not install use ``pip install mlrun==<mlrun server version>`` or ``sh align_mlrun.sh`` (our default mlrun installer - automatically install the server version)

In [1]:
import mlrun

#### Load or Create a new project Based on an exciting project.yaml file
For loading a project you can use :

MLRun provides you three option to create or loading a project:

1. [get_or_create_project](https://docs.mlrun.org/en/latest/api/mlrun.projects.html?highlight=get_or_create_project#mlrun.projects.get_or_create_project) - this method allows you to load a project from MLRun DB or optionally load it from a yaml/zip/tar/git template, and create a new project if doesnt exist
2. [new_project](https://docs.mlrun.org/en/latest/api/mlrun.projects.html?highlight=new_project#mlrun.projects.new_project) - Create a new MLRun project, optionally load it from a yaml/zip/tar/git template.
3. [load_project](https://docs.mlrun.org/en/latest/api/mlrun.projects.html?highlight=load_project#mlrun.projects.load_project) - Load an MLRun project from yaml/zip/tar/git/dir or from MLRun db


**Note-** If you want to run your code locally you must clone the files to the context directory, execute automaticlly when you load project or create new project and define a url params

In [2]:
#project source to load from
source = 'git://github.com/GiladShapira94/example-ci-cd.git#master'

**Note -** Please add git branch or refs to the source e.g.: 'git://<url>/org/repo.git#<branch-name or refs/heads/..>'

In [3]:
project = mlrun.load_project('./clone',url=source,clone=True,name='my-load-proj')

**context -** project local directory path (default value = "./")

**url -**     name (in DB) or git or tar.gz or .zip sources archive

**clone -**   if True, always clone (delete any existing content)

**name -**    project name

By Specifying a url/remote address MLRun automatically clone/extract your git repository ot tar/zip file

**Secrets -** for working with private repo you need to provide a Gittoken as a secrets using mlrun or mlrun UI, for example:
```
#while loading from private repo
project = mlrun.get_or_create_project(name='new-git-proj',context='./',init_git=True,secrest={"GIT_TOKEN":<github-token>})
#while running functions in a project from a private repo
project.set_secrets({"GIT_TOKEN":<github-token>}
```

In [4]:
#print project yaml
print(project.to_yaml())

kind: project
metadata:
  name: my-load-proj
spec:
  functions:
  - url: ./src/data_fetch.py
    name: data-fetch
    kind: job
    image: mlrun/mlrun
    handler: data_fetch
    with_repo: true
  - url: ./src/train.py
    name: train
    kind: job
    image: mlrun/mlrun
    handler: train
    with_repo: true
  - url: ./src/serving.yaml
    name: serving
  workflows:
  - path: ./src/workflow.py
    name: main
  artifacts:
  - kind: model
    metadata:
      project: new-git-proj
      key: model-test
    spec:
      target_path: v3io:///projects/new-git-proj/artifacts/a668c9a7-27a2-446c-84fa-8e93fbd75ac7/train/0/model/
      model_file: model.pkl
    status:
      state: created
  conda: ''
  source: git://github.com/GiladShapira94/example-ci-cd.git#refs/heads/master
  origin_url: git://github.com/GiladShapira94/example-ci-cd.git#refs/heads/master
  load_source_on_run: true
  desired_state: online



#### Run Function 
For run a function you will need to use the [run_function](https://docs.mlrun.org/en/latest/api/mlrun.projects.html?highlight=run_function#mlrun.projects.MlrunProject.run_function) method.
This method allows you to run a MLRun jobs locally and remotely as long as there is no requirments ( if there is any requirments you will need to build a new image before you run a function)
It is equivalent to func.run method.

In [5]:
project.run_function(function='data-fetch',local=True,returns=['train-dataset','test-dataset'])

> 2023-04-09 11:28:28,255 [info] Storing function: {'name': 'data-fetch-data-fetch', 'uid': 'd77cf9ca7dc8488f8d0f3ade12fa063a', 'db': 'http://mlrun-api:8080'}


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
my-load-proj,...12fa063a,0,Apr 09 11:28:28,completed,data-fetch-data-fetch,v3io_user=shapirakind=owner=shapirahost=jupyter-shapira-7fc985f9db-r8lqk,,,,train-datasettest-dataset


> 2023-04-09 11:28:41,594 [info] run executed, status=completed: {'name': 'data-fetch-data-fetch'}


In [6]:
data_fetch_run = project.run_function(function='data-fetch',local=False,returns=['train-dataset','test-dataset'])

> 2023-04-09 11:28:41,635 [info] Storing function: {'name': 'data-fetch-data-fetch', 'uid': '1b94d4f4356b49a9a9adc83c5fdb19d5', 'db': 'http://mlrun-api:8080'}
> 2023-04-09 11:28:41,856 [info] Job is running in the background, pod: data-fetch-data-fetch-rs445
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
my-load-proj,...5fdb19d5,0,Apr 09 11:28:45,completed,data-fetch-data-fetch,v3io_user=shapirakind=jobowner=shapiramlrun/client_version=1.3.0mlrun/client_python_version=3.7.6host=data-fetch-data-fetch-rs445,,,,train-datasettest-dataset


> 2023-04-09 11:28:55,054 [info] run executed, status=completed: {'name': 'data-fetch-data-fetch'}


In [7]:
train_run = project.run_function(function='train',inputs={'train_data':data_fetch_run.outputs['train-dataset'],'test_data':data_fetch_run.outputs['test-dataset']})

> 2023-04-09 11:28:55,187 [info] Storing function: {'name': 'train-train', 'uid': 'c5adfcfb259949a5a181b74eef6ef583', 'db': 'http://mlrun-api:8080'}
> 2023-04-09 11:28:55,426 [info] Job is running in the background, pod: train-train-7ds5c
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
my-load-proj,...ef6ef583,0,Apr 09 11:28:59,completed,train-train,v3io_user=shapirakind=jobowner=shapiramlrun/client_version=1.3.0mlrun/client_python_version=3.7.6host=train-train-7ds5c,train_datatest_data,,accuracy=0.85f1_score=0.888888888888889precision_score=0.8571428571428571recall_score=0.9230769230769231,feature-importancetest_setconfusion-matrixroc-curvescalibration-curvemodel


> 2023-04-09 11:29:16,617 [info] run executed, status=completed: {'name': 'train-train'}


#### Get Function Object 
To get function object you will need to use the [get_function](https://docs.mlrun.org/en/latest/api/mlrun.projects.html?highlight=get_function#mlrun.projects.MlrunProject.get_function) method.
This method allows you to get a function object based on the metadata in your project YAML file.
````
serving_func = project.get_function('<function name>')
````

In [8]:
serving_func = project.get_function('serving')

**Note -** this serving function is pointing to a model file, this model path value is stored in the function spec if you want to change it (maybe you want to use a newer model file) you need to add the model to the function object and then deploy the function or change the function.yaml in the remote source, see below example:
```
serving_func = project.get_function('serving')
serving_func.add_model(key='model',model_path=train_run.outputs["model"], class_name='mlrun.frameworks.sklearn.SklearnModelServer')
serving_dep = project.deploy_function('serving')
```

Test your serving funciton locally before deploying it

In [9]:
serving_server = serving_func.to_mock_server()

> 2023-04-09 11:29:16,726 [warning] run command, file or code were not specified
> 2023-04-09 11:29:18,865 [error] Traceback (most recent call last):
  File "/User/.pythonlibs/jupyter-shapira/lib/python3.7/site-packages/mlrun/errors.py", line 94, in raise_for_status
    response.raise_for_status()
  File "/User/.pythonlibs/jupyter-shapira/lib/python3.7/site-packages/requests/models.py", line 1021, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 404 Client Error: Not Found for url: http://mlrun-api:8080/api/v1/projects/new-git-proj/artifacts/model?tag=2a475cd976cf4aba907c52c1f9235604&format=full

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/User/.pythonlibs/jupyter-shapira/lib/python3.7/site-packages/mlrun/datastore/datastore.py", line 154, in get_store_artifact
    data_store_secrets=secrets,
  File "/User/.pythonlibs/jupyter-shapira/lib/python3.7/site-packages/mlr

RuntimeError: failed to load model model

In [ ]:
my_data = '''{"inputs":[[-0.60150011,  0.51150308,  0.25701239, -1.51777297, -1.82961288,
         0.22983693, -0.40761625,  0.82325082,  1.1779216 ,  1.08424275,
        -0.7031145 , -0.40608979, -0.36305977,  1.28075006,  0.94445967,
         1.19105828,  1.93498414,  0.69911167,  0.50759757,  0.91565635]]}'''

In [ ]:
serving_server.test('/',my_data)

#### Deploy Function
For deplying remote function as nuclio or serving you will need to use the [deploy_function](https://docs.mlrun.org/en/latest/api/mlrun.projects.html?highlight=deploy_function#mlrun.projects.MlrunProject.deploy_function) method.
You must use this method before invoke nuclio or serving fucntions.
It is equivalent to func.deploy() method.
````
nuclio_func=project.deploy_function(function='<function name>')

nuclio_func.function.invoke('/',{'int':4})
````

In [ ]:
serving_dep = project.deploy_function('serving')

In [ ]:
serving_dep.function.invoke('/',5)

#### Run a workflow

In [ ]:
# run workflow named main and wait for pipeline completion (watch=True)
project.run('main',watch=True)

**Run Schedule workflow**

For more inforation about scheduling workflows, please check this [link](https://docs.mlrun.org/en/latest/concepts/scheduled-jobs.html)

In [ ]:
project.run('main',watch=True,schedule='0 * * * *')